In [1]:
import sys
sys.path.append('../')
import nexullance.LP_optimal as LP_optimal
# import nexullance.LP_gurobi as LP_MP
import topologies.Slimfly as Slimfly
import globals as gl
import time
import tracemalloc
from statistics import mean
import networkx as nx
import pynauty as nauty

In [2]:
config = (98, 11)
topo_name='slimfly'
EPR=config[1]//2
_network = Slimfly.Slimflytopo(config[0], config[1])
V=config[0]
assert(V==_network.nx_graph.number_of_nodes())
network_edges=list(_network.nx_graph.edges())

In [3]:
p2p_TM=gl.generate_shift_traffic_pattern(config[0], EPR)
R2R_TM=gl.convert_p2p_traffic_matrix_to_R2R(p2p_TM, config[0], EPR)

In [4]:
#initialize adjacency dict to that of G_1
G_adjacency_dict={n: list(nbrdict.keys()) for n, nbrdict in _network.nx_graph.adjacency()}
vertex_weight={} #first a dictionary of vertex weights, later converted to a list of sets
G_2_edges=0
for i in range(V):
    for j in range(V):
        if R2R_TM[i][j] > 0:
            new_v_id=V+G_2_edges
            G_adjacency_dict[new_v_id]=[i]
            G_adjacency_dict[j].append(new_v_id)
            if R2R_TM[i][j] in vertex_weight.keys():
                assert(new_v_id not in vertex_weight[R2R_TM[i][j]])
                vertex_weight[R2R_TM[i][j]].append(new_v_id)
            else:
                vertex_weight[R2R_TM[i][j]]=[new_v_id]
            G_2_edges+=1

# convert vertex weight dict to a list of color sets
G_color_sets=[]
for color_set in vertex_weight.values():
    G_color_sets.append(set(color_set))

In [5]:
G=nauty.Graph(V+G_2_edges, directed=True, adjacency_dict=G_adjacency_dict, vertex_coloring=G_color_sets)

In [6]:
gen_set=gl.nauty_autgrp_verbose(G, _verbose=True)

Generators of the automorphism group:
[[0, 1, 2, 3, 4, 5, 6, 42, 43, 44, 45, 46, 47, 48, 35, 36, 37, 38, 39, 40, 41, 28, 29, 30, 31, 32, 33, 34, 21, 22, 23, 24, 25, 26, 27, 14, 15, 16, 17, 18, 19, 20, 7, 8, 9, 10, 11, 12, 13, 49, 50, 51, 52, 53, 54, 55, 91, 92, 93, 94, 95, 96, 97, 84, 85, 86, 87, 88, 89, 90, 77, 78, 79, 80, 81, 82, 83, 70, 71, 72, 73, 74, 75, 76, 63, 64, 65, 66, 67, 68, 69, 56, 57, 58, 59, 60, 61, 62, 98, 99, 100, 101, 102, 103, 104, 140, 141, 142, 143, 144, 145, 146, 133, 134, 135, 136, 137, 138, 139, 126, 127, 128, 129, 130, 131, 132, 119, 120, 121, 122, 123, 124, 125, 112, 113, 114, 115, 116, 117, 118, 105, 106, 107, 108, 109, 110, 111, 147, 148, 149, 150, 151, 152, 153, 189, 190, 191, 192, 193, 194, 195, 182, 183, 184, 185, 186, 187, 188, 175, 176, 177, 178, 179, 180, 181, 168, 169, 170, 171, 172, 173, 174, 161, 162, 163, 164, 165, 166, 167, 154, 155, 156, 157, 158, 159, 160], [1, 2, 3, 4, 5, 6, 0, 8, 9, 10, 11, 12, 13, 7, 15, 16, 17, 18, 19, 20, 14, 22, 23, 24, 25

In [7]:
# apply automorphism on a vector
def apply_automorphism(vector, automorphism):
    new_vector=[]
    for v in vector:
        new_vector.append(automorphism[v])
    return new_vector
        

# find representative vector by the automorphism group
def representative_vector(vectors, generator_set):
    # inputs: 
    # 1. vectors: a list of vectors of the same size '_n', each vector is in the space of V^n, V is the number of vertices in G_1
    # 2. generator: generator set of the automorphism group
    num_vectors=len(vectors)
    _n=len(vectors[0])

    # outputs:
    # 1. representative vectors
    # 2. the exact automorphisms that maps all vectors to their representatives
    rep_vectors_to_all={} # keys: representative vectors, values: represented vectors

    # initialzing necessary data structures:
    orbit=set()
    checked_vectors=set()

    # start of the algorithm:
    for vector in vectors:
        if tuple(vector) in checked_vectors:
            continue
        representative=tuple(vector)
        rep_vectors_to_all[representative]=[]
        orbit.add(tuple(vector))
        while len(orbit)>0:
            _v=orbit.pop()
            checked_vectors.add(_v)
            for gen in generator_set:
                _w=apply_automorphism(_v, gen)
                if tuple(_w) not in checked_vectors:
                    orbit.add(tuple(_w))
                    rep_vectors_to_all[representative].append(_w)

    return rep_vectors_to_all



In [8]:
vectors=[]
for s in range(V):
    for d in range(V):
        for (i, j) in network_edges:
            vectors.append([s, d, i, j])
            vectors.append([s, d, j, i])

In [9]:
len(vectors)

10353112

In [10]:
representative_dict=representative_vector(vectors, gen_set)

In [11]:
len(representative_dict)

740488

In [12]:
len(representative_dict)/len(vectors)

0.07152322895763129